In [1]:
import csv
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
import pickle as pkl
import matplotlib.pyplot as plt
import random
from pymer4.models import Lmer as lm
import itertools as it
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings('ignore')

In [2]:
gdp_regression_data_insample = pd.read_csv("../data/regression/cross_validation/gdp_regression_data_insample_festratified_0.csv")

In [3]:
model_spec = {
    "continuous_covariates" : [
        'precip_annual_std_[weight]',
        'precip_annual_std_[weight]_2',
        'precip_annual_std_[weight]_3',
        'temp_[weight]',
        'temp_[weight]_2', 
        'temp_[weight]_3'
    ],
    "discrete_covariates" : ['drought'],
    "fixed_effects" : ["year"],
    "incremental_effects" : 0,
    "weights" : "pop_weighted",
    "target" : "fd_ln_gdp"
}

In [10]:
vars_to_grab = []
for covar in model_spec["continuous_covariates"]:
    vars_to_grab.append(covar.replace("[weight]",model_spec["weights"]))
for i in range(model_spec["incremental_effects"]):
    for ie_col in [col for col in gdp_regression_data_insample.columns if col.endswith(f"incremental_effect_{i+1}")]:
        vars_to_grab.append(ie_col)

In [11]:
%load_ext rpy2.ipython
formula = model_spec["target"] + " ~ " + " + ".join(vars_to_grab) + " + (1|year) + (0 + drought | country)"
%%R -i gdp_regression_data_insample -i formula
library(lme4)
model = lmer(formula, data=gdp_regression_data_insample)

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
fd_ln_gdp ~ precip_annual_std_pop_weighted + precip_annual_std_pop_weighted_2 + precip_annual_std_pop_weighted_3 + temp_pop_weighted + temp_pop_weighted_2 + temp_pop_weighted_3 + (1|year) + (0 + drought | country)
